In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Embedding, MaxPooling1D, GlobalMaxPooling1D, LSTM, Dropout,SimpleRNN,Bidirectional,Attention
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import re

In [ ]:
MAX_SEQUENCE_LENGTH = 300
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 200

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AraSenCorpus/Balanced/tweets.txt', names=['Class','Text'], sep='\t')
df
df.groupby('Class').count()
#, skiprows=1

In [ ]:
sentences = df['Text'].values.astype(str)
target = df['Class'].map({'Neutral':0,'Positive':1,'Negative':2}).values

In [ ]:
target_cat = to_categorical(target, num_classes=3)

In [ ]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word2idx = tokenizer.word_index #dictionary of all unique words in our document

In [ ]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)
num_words = min(MAX_VOCAB_SIZE,len(word2idx)+1)
print(MAX_SEQUENCE_LENGTH)
print(num_words)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data,target_cat,test_size=0.2, shuffle=True, random_state=98,stratify=target_cat)

In [ ]:
#LSTM
model2 = Sequential()

In [ ]:
model2.add(Embedding(num_words,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH,trainable=True,input_shape=(MAX_SEQUENCE_LENGTH,)))
model2.add(LSTM(1024,return_sequences=True))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(6,activation='softmax'))
model2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=3,mode='min')
model2.fit(X_train,y_train,batch_size=128,epochs=1,validation_data=(X_test,y_test),callbacks=[early_stop])

In [ ]:
pred2 = model2.predict_classes(X_test)

In [ ]:
def original_values(num):
    if num==0:
        return 'Neutral'
    elif num==1:
        return 'Positive'
    elif num==2:
        return 'Negative'   

In [ ]:
#convert 0,1,2 back to original form 
pred = np.array([original_values(num) for num in pred2])
y_test = np.argmax(y_test,axis=-1)
y_true = np.array([original_values(num) for num in y_test])
print(classification_report(y_true,pred))